In [1]:
import torch
from multimodal_particles.models import AbsorbingFlow
from multimodal_particles.config_classes.absorbing_flows_config import AbsorbingConfig
from multimodal_particles.data.particle_clouds.jets import JetDataclass
from multimodal_particles.data.particle_clouds.jets_dataloader import JetsDataloaderModule
from multimodal_particles.models.generative.bridges import AbsorbingBridge

In [2]:
# Define parameters
config = AbsorbingConfig()
t = torch.tensor([0.5,0.4, 1.0], dtype=torch.float32)  # Example input (B,)

config = AbsorbingConfig()
model = AbsorbingFlow(config)
random_databatch = JetsDataloaderModule.random_databatch(config)

/home/cesarali/particles/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [4]:
import torch
import torch.nn as nn

# Example logits output from your model (B, 1)
logits = torch.tensor([[0.5], [-1.2], [2.0], [0.0]])  # Shape (B, 1)

# Corresponding targets (B, 1), should be {0, 1}
targets = torch.tensor([[1], [0], [1], [0]]).float()  # Shape (B, 1)

# Define the loss function
loss_fn = nn.BCEWithLogitsLoss()

# Calculate the loss
loss = loss_fn(logits, targets)
print(f"Loss: {loss.item()}")


Loss: 0.38935863971710205
